### 한빛 아카데비 새로나온책, 서브페이지 정보까지

    * 제목, 저자, 번역, 출간일 ,쪽수, 판매가

https://www.hanbit.co.kr/academy/books/new_book_list.html?page=2&cate_cd=&srt=&searchKey=&keyWord=

In [2]:
import requests
from bs4 import BeautifulSoup
import re  # 정규표현식
import pandas as pd
from tqdm.notebook import tqdm
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
base_url = "https://www.hanbit.co.kr/"
p = [str(i+1) for i in range(11)]   


In [28]:
line = []

for i in tqdm(range(len(p))):   # 페이지 loop

    # .. main 링크 페이지
    url_main = base_url + f"/academy/books/new_book_list.html?page={p[i]}&cate_cd=&srt=&searchKey=&keyWord="
    html_main = requests.get(url_main, headers = headers).text
    soup_main = BeautifulSoup(html_main, 'html.parser')
    #soup_main

    books = soup_main.select('div.sub_book_list_area > li')

    for k in range(len(books)): # 페이지당 책리스트

        # 서브링크 및 box 정보
        url_sub = base_url + books[k].select_one('p.book_tit > a')['href']
        #url_sub

        html_sub = requests.get(url_sub, headers = headers).text
        soup_sub = BeautifulSoup(html_sub, 'html.parser')
        #soup_sub

        bestList_sub = soup_sub.select('div.store_view_area')
        bestList_sub

        title, author, transl, publdt, pagenb = '---', '---', '---', '---', '---'

        for j in range(len(bestList_sub)):
            # 제목
            title = bestList_sub[j].select_one('h3').text

            # info list: 저자, 번역, 출간 , 페이지
            info_list = bestList_sub[j].select('ul.info_list > li')

            for li in info_list:
                text = li.get_text().strip()
                #print(text)
                if '저자 :'   in text: author = text.split(':')[1].strip()
                if '번역 :'   in text: transl = text.split(':')[1].strip()
                if '페이지 :' in text: pagenb = text.split(':')[1].strip()
                if '출간 :'   in text: publdt = text.split(':')[1].strip()
                #print(author, transl, pagenb, publdt)

            # 판매가
            price_n = int(soup_sub.select_one('span.pbr > strong').text.replace(',',""))

            line.append( [title, author, transl, publdt, pagenb, price_n] )

df = pd.DataFrame(line, columns=['제목', '저자', '번역', '페이지', '출간', '판매가'])
df



  0%|          | 0/11 [00:00<?, ?it/s]

,제목,저자,번역,페이지,출간,판매가
0,처음 배우는 매트랩,방성완,---,2023-04-10,416 쪽,30000
1,SAS와 메타분석을 활용한 한번에 적용하는 의학보건통계,히든그레이스 데이터분석팀,---,2023-03-09,516 쪽,34000
2,정보교과교육론(3판),"최현종 , 전용주",---,2023-03-01,488 쪽,30000
3,"IT CookBook, 기초전자실험 with PSpice(3판)",홍순관,---,2023-02-28,308 쪽,22000
4,경제수학 강의(3판),김성현,---,2023-02-28,464 쪽,23000
...,...,...,...,...,...,...
215,실무로 완성하는 PLC(멜섹Q) : 서보를 이용한 위치 제어,정완보,---,2020-01-02,568 쪽,28000
216,"College Physics, 5th Edition",Alan Giambattista,---,2020-01-01,1232 쪽,52000
217,Stata로 끝내는 논문,정규형,---,2019-12-09,372 쪽,25000
218,Business Statistics: For Contemporary Decision...,Ken Black,---,2019-12-01,832 쪽,49000


In [26]:
df.to_csv('../daou/한빛서브out')